# Import needed modules

In [1]:
from agents import *
from random import choice
import numpy as np
import sys
import collections

# Only if needed for new python versions
if sys.version_info >= (3, 10):
    import collections
    collections.Iterable = collections.abc.Iterable
    collections.Sequence = collections.abc.Sequence

# Define our Agenet Class
> The agent is a vacum cleaner

Has the following capabilities:

- Whathever

Old move:
    
```python

def moveforward(self, success=True):
    '''moveforward possible only if success (i.e. valid destination location)'''
    if not success:
        return
    if self.direction.direction == Direction.R:
        self.location[0] += 1
    elif self.direction.direction == Direction.L:
        self.location[0] -= 1
    elif self.direction.direction == Direction.D:
        self.location[1] += 1
    elif self.direction.direction == Direction.U:
        self.location[1] -= 1
```

In [2]:
class Roomba(Agent):
    location = [1,1]
    direction = Direction("down")

    def MoveForward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location = (self.location[0] + 1, self.location[1])
        elif self.direction.direction == Direction.L:
            self.location = (self.location[0] - 1, self.location[1])
        elif self.direction.direction == Direction.D:
            self.location = (self.location[0], self.location[1] + 1)
        elif self.direction.direction == Direction.U:
            self.location = (self.location[0], self.location[1] - 1)

    def Turn(self, d):
        self.direction = self.direction + d
        
    def Suck(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Dirt):
            return True
        return False

# Enviroment
> Defining a Vacumm Cleaner environment

Our vacum cleaner envirnment has blah blah balh

- Good 
- Awsome
- and everythign else

---

TODO:

- [ ] No objects just walls
- [X] Add Obstacle, almost correct placement still need fixes though
- [X] Make obstacle work
- [ ] Add walls in the corners if there are walls near it in both direction
- [ ] Collision needs to be fixed (actually we need to change the simplerlex agent program to not be so stupid) 
- You can litteraly use that since it is already implemented in agents

In [199]:
class Roomba_Env(XYEnvironment):
    # Define members of the class Roomba_Env
    
    # Probability to spawn a pit in a location
    dirt_probability = random.uniform(0.1, 0.3)
    # Probability to spawn an obstacle in a location
    obstacle_probability = random.uniform(0.1, 0.9) 
    # Do not print the movment of the Rumba 
    verbose = False 
    
    # Define the global variable for the colors
    colors = {
        #'Obstacle': (80, 80, 80),
        'Obstacle': (44, 53, 57),
        'Wall': (44, 53, 57),
        'Roomba': (200, 0, 0),
        'Dirt': (125, 100, 80)}
    
    # Room should be 8x8 grid of rooms. The extra 3 for walls
    # Maybe I will just do random size + 3 in the future
    def __init__(self, random = True, width=9, height=9):
        if random == True:
            # Generate random dimensions between 9 and 15
            width = np.random.randint(9, 15)
            height = np.random.randint(9, 15)
        
        # Call the above class with the custom size
        super().__init__(width, height)
        
        # Creates a grid
        self.grid = BlockGrid(width, height, fill=(200, 200, 200))
        self.bounded = True
        
        self.init_world()
           
    def add_walls(self):
        """Put walls around the entire perimeter of the grid."""
        num_layers = 2
        for layer in range(num_layers):
            for x in range(layer, self.width - layer):
                if layer == 0 or random.random() < (num_layers - layer) / num_layers:
                    self.add_thing(Wall(), (x, layer))
                if layer == 0 or random.random() < (num_layers - layer) / num_layers:
                    self.add_thing(Wall(), (x, self.height - 1 - layer))
                    
            for y in range(layer + 1, self.height - 1 - layer):
                if layer == 0 or random.random() < (num_layers - layer) / num_layers:
                    self.add_thing(Wall(), (layer, y))
                if layer == 0 or random.random() < (num_layers - layer) / num_layers:
                    self.add_thing(Wall(), (self.width - 1 - layer, y))
                    
        # Add walls at second layer corners if there are walls near those corners
        # someone needs to add this code

    def add_obstacles(self, x_bounds, y_bounds):
        """Add objects inside grid that work like walls."""
        
        # first 4 all essentialy because we do not want chess grid like walls 
        # The second 2 are becuase we do not want full lines of walls + obstacles which block the agent
        surroundings = [(1 , 1), (-1, -1), (-1, 1), (1, -1), (1, 0), (0, 1)]
        
        # Loops to add the obstacles
        for x in range(*x_bounds):
            for y in range(*y_bounds):
                if random.random() < self.obstacle_probability:
                    # Rivedi  
                    # Check if the location is not a wall and has at least one free adjacent cell in every direction of movement
                    if all(self.is_inbounds((x + dx, y + dy)) and \
                        not any((isinstance(thing, Wall) or isinstance(thing, Obstacle)) for thing in self.list_things_at((x + dx, y + dy))) for dx, dy in surroundings):
                        self.add_thing(Obstacle(), (x, y), True)
                        
    def add_dirt(self, x_bounds, y_bounds):
        """Add dirt inside grid"""
        for x in range(*x_bounds):
            for y in range(*y_bounds):
                if random.random() < self.dirt_probability and not any((isinstance(thing, Obstacle) or (isinstance(thing, Wall))) for thing in self.list_things_at((x, y))):
                    # and not any(isinstance(thing, Wall) for thing in self.list_things_at((x, y))):
                    # Then you can add the dirt
                    self.add_thing(Dirt(), (x, y), True)
                    
    def init_world(self,):
        """Spawn items in the world based on probabilities from the book"""
        # Bounds for the objects and dirt to not overlap with the walls
        x_bounds = [1, self.x_end - 1]
        y_bounds = [1, self.y_end - 1]
    
        # add walls objects and dit 
        self.add_walls()
        self.add_obstacles(x_bounds, y_bounds)
        self.add_dirt(x_bounds, y_bounds)

        
    def get_world(self):
        """Returns all the items in the world in a format
        understandable by the ipythonblocks BlockGrid."""
        result = []
        x_start, y_start = (0, 0)
        x_end, y_end = self.width, self.height

        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                row.append(self.list_things_at((x, y)))
            result.append(row)
            
        return result
    
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        loc = list(copy.deepcopy(agent.location))  # loc is now a list
        # find out the target location
        # Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(tuple(loc)):  # convert loc back to a tuple
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        agent.bump = False
        if action == 'TurnRight':
            if self.verbose == True:
                print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.Turn(Direction.R)
            agent.performance -= 1
        elif action == 'TurnLeft':
            if self.verbose == True:
                print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.Turn(Direction.L)
            agent.performance -= 1
        elif action == 'MoveForward':
            if self.verbose == True:
                print('{} decided to Move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.MoveForward()
            agent.performance -= 1
        elif action == "Suck":
            items = self.list_things_at(agent.location, tclass=Dirt)
            if len(items) != 0:
                if agent.Suck(items[0]):
                    if self.verbose == True:
                        print('{} suck {} at location: {}, :)' .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    agent.performance += 100
    
    def run(self, steps=100, delay=0, draw=False):
        """Run the Environment for given number of time steps,
        but update the GUI too."""
        for step in range(steps):
            self.update(delay, draw)
            if self.is_done():
                break
            self.step()
        self.update(delay,draw)
        
    def update(self, delay=0, draw=False):
        sleep(delay)
        self.reveal(draw)

    def reveal(self, draw=False):
        """Display the BlockGrid for this world - the last thing to be added
        at a location defines the location color."""
        self.draw_world()
        # apply changes to the same grid instead
        # of making a new one.
        if draw==True:
            clear_output(1) # This shows the enviroment
            self.grid.show()
        # self.visible = False
        
    def draw_world(self):
        self.grid[:] = (200, 200, 200)
        world = self.get_world()
        for x in range(0, len(world)):
            for y in range(0, len(world[x])):
                if len(world[x][y]):
                    self.grid[y, x] = self.colors[world[x][y][-1].__class__.__name__]
                    
    def is_done(self):
        '''Is done if ther is no more dirt to Suck'''
        # There is no dirt left
        no_dirt = False 
        if not any(isinstance(thing, Dirt) for thing in self.things):
            print("There is not dirt left")
            no_dirt = True 
        # Return if the game is ended
        return no_dirt

## Define the agant program
> Silly reflex agent movments

It has the following moves: 

- Attack 
- Just kidding

---

Then use the agent in the rumba enviroment for a run

In [200]:
def simple_program(percepts):
    '''Returns an action based on it's percepts'''
    for p in percepts: # Duable actions
        if isinstance(p, Dirt):
            return 'Suck'
        
        # This still bumbs into walls because it takes action which might still lead to bumping into a wall
        if isinstance(p, Bump): # then check if it bumbs agains something 
            turn = False
            choice = random.choice((1,2));
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
    if choice == 1:
        return 'TurnRight'
    elif choice == 2:
        return 'TurnLeft'
    else:
        return 'MoveForward'

### Finally run the enviroment once

In [205]:
# Create the enviroment with the correct program
room = Roomba_Env()

import random

# Define a function to get a random position that is not on top of walls, dirt, or obstacles
def get_random_position(room):
    while True:
        x = random.randint(1, room.width - 2)
        y = random.randint(1, room.height - 2)
        # Good way to write the if statement
        objects = [Wall, Dirt, Obstacle]
        if not any(isinstance(obj, var) for var in objects for obj in room.list_things_at((x, y))):
            return (x, y)

# Add the Roomba object to the environment at a random position
room.add_thing(Roomba(simple_program), get_random_position(room), True)

# Run the rumba for x step with 0 delay because we want to be fast but still show the cool drawings
room.run(steps = 5, delay = 0, draw=True)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


## Using the test agent function
> The definition can be found inside agents.py

In [6]:
# Import for progress bar
from tqdm import tqdm

# Custom implementation to allows the agent to be placed in a random position
def test_agent(AgentFactory, steps, envs):
    def compute_score(env):
        agent = AgentFactory()
        env.add_thing(agent, get_random_position(room), True) # maybe
        # env.add_thing(agent) # maybe it already places in a correct location questionable
        env.run(steps)
        return agent.performance
    
    def mean_score_with_progress(envs):
        scores = []
        for env in tqdm(envs):
            score = compute_score(env)
            scores.append(score)

        return mean(scores)

    return mean_score_with_progress(envs)

steps = 100
n_envs = 100

test_agent(lambda: Roomba(simple_program), steps, [Roomba_Env() for _ in range(n_envs)])

 37%|███████████████████████████████████████████████████████████▉                                                                                                      | 37/100 [00:00<00:00, 124.06it/s]

There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left


 63%|██████████████████████████████████████████████████████████████████████████████████████████████████████                                                            | 63/100 [00:00<00:00, 118.58it/s]

There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 99/100 [00:00<00:00, 111.43it/s]

There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left
There is not dirt left


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 112.21it/s]


0